<a href="https://colab.research.google.com/github/MaksimSmolenkov/NumMethods/blob/main/%D0%A7%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 Лабораторная 16 вариант

In [ ]:
import numpy as np
import math
m = 16

# Метод Гаусса

In [ ]:
def gaussian_elimination(A, b):
    n = len(b)

    for i in range(n):
        # Находим максимальный элемент в столбце
        max_row = i + np.argmax(np.abs(A[i:, i]))
        # Меняем местами текущую строку и строку с максимальным элементом
        A[[i, max_row]] = A[[max_row, i]]
        b[i], b[max_row] = b[max_row], b[i]

        # Обнуляем элементы под текущим элементом главной диагонали
        for j in range(i + 1, n):
            factor = A[j][i] / A[i][i]
            A[j] -= factor * A[i]
            b[j] -= factor * b[i]

    # Обратный ход
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = round((b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i][i])

    return x


In [ ]:
A = np.array([[1, 1, 1, 0],
              [1, 2, 2, 2],
              [2, 3, 4, 4],
              [3, 4, 5, 6]], dtype=float)

b = np.array([0, 1, 1, 1], dtype=float)

solution = gaussian_elimination(A, b)
print("Решение:", solution)

Решение: [-1.  1.  0.  0.]


# Метод обратной матрицы

In [ ]:
import numpy as np

def solve_with_inverse(A, b):
    # Проверяем, является ли матрица A квадратной
    if A.shape[0] != A.shape[1]:
        raise ValueError("Матрица A должна быть квадратной.")

    # Находим обратную матрицу
    try:
        A_inv = np.linalg.inv(A)
    except np.linalg.LinAlgError:
        raise ValueError("Матрица A не имеет обратной матрицы (вырождена).")

    # Находим решение
    x = np.dot(A_inv, b)
    x = np.round(x).astype(int)
    return x

In [ ]:
solution = solve_with_inverse(A, b)
solution.astype(int)
print("Решение:", solution)

Решение: [-1  1  0  0]


## Метод Зейделя

In [ ]:
def seidel(A, b, x0=None, tolerance=1e-10, max_iterations=1000):
    n = len(b)

    # Начальное приближение
    if x0 is None:
        x0 = np.zeros(n)

    x = np.copy(x0)

    for iteration in range(max_iterations):
        x_old = np.copy(x)

        for i in range(n):
            # Вычисляем сумму для текущего уравнения
            sigma = sum(A[i][j] * x[j] for j in range(n) if j != i)
            # Обновляем значение x[i]
            x[i] = (b[i] - sigma) / A[i][i]

        # Проверяем условие сходимости
        if np.linalg.norm(x - x_old, ord=np.inf) < tolerance:
            print(f"Сошлось за {iteration + 1} итераций.")
            return x

    raise ValueError("Метод не сошелся за максимальное число итераций.")

In [ ]:
A = np.array([[3, 1, -1, 1],
              [1, -4, 1, -1],
              [-1, 1, 4, 1],
              [1, 2, 1, -5]], dtype=float)

b = np.array([3*m, m-6, 15-m, m+2], dtype=float)

solution = seidel(A, b)
print("Решение:", solution)

Сошлось за 21 итераций.
Решение: [16.  2.  3.  1.]


# Метод Якоби

In [ ]:
def jacobi(A, b, x0=None, tolerance=0.01, max_iterations=1000):
    n = len(b)

    # Начальное приближение
    if x0 is None:
        x0 = np.zeros(n)

    x = np.copy(x0)

    for iteration in range(max_iterations):
        x_old = np.copy(x)

        for i in range(n):
            # Вычисляем сумму для текущего уравнения
            sigma = sum(A[i][j] * x_old[j] for j in range(n) if j != i)
            # Обновляем значение x[i]
            x[i] = (b[i] - sigma) / A[i][i]

        # Проверяем условие сходимости
        if np.linalg.norm(x - x_old, ord=np.inf) < tolerance:
            print(f"Сошлось за {iteration + 1} итераций.")
            return x

    raise ValueError("Метод не сошелся за максимальное число итераций.")

In [ ]:
solution = jacobi(A, b)
print("Решение:", solution)

Сошлось за 15 итераций.
Решение: [15.99776066  1.99621605  2.99796596  0.99691651]


# Метод прогонки

In [ ]:
def tridiagonal_solver(a, b, c, d):
    """
    Решает систему линейных уравнений Ax = d, где A - трёхдиагональная матрица.

    Параметры:
    a : array-like
        Поддиагональные элементы (длина n-1).
    b : array-like
        Диагональные элементы (длина n).
    c : array-like
        Наддиагональные элементы (длина n-1).
    d : array-like
        Правая часть (длина n).

    Возвращает:
    x : array
        Решение системы линейных уравнений.
    """
    n = len(d)

    # Прямой ход
    for i in range(1, n):
        w = a[i-1] / b[i-1]
        b[i] -= w * c[i-1]
        d[i] -= w * d[i-1]

    # Обратный ход
    x = np.zeros(n)
    x[-1] = d[-1] / b[-1]

    for i in range(n-2, -1, -1):
        x[i] = (d[i] - c[i] * x[i+1]) / b[i]

    return x

In [ ]:
a = np.array([2, 1.1, 5, -2])  # Поддиагональные элементы (n-1)
b = np.array([-3, -5, 4, 9, 6.5])  # Диагональные элементы (n)
c = np.array([1.2, 1, -1, 2])  # Наддиагональные элементы (n-1)
d = np.array([-1.7, -2, 3, 11, 2])  # Правая часть

solution = tridiagonal_solver(a, b, c, d)
print("Решение:", solution)

Решение: [0.92999403 0.9083184  0.68160393 0.72556596 0.53094337]


# 2 Часть

In [ ]:
A = ([[-7.46, 11.24, -0.54, 14.31],
                [0.16, 17.15, -1.11, 10.16],
                [23.17, 10.26, 1.19, 7.43],
                [-28.16, 0.15, 0.97, -3.41]])


eigenvalues, eigenvectors = np.linalg.eig(A)

print("Собственные векторы: ",eigenvectors )
print("Собственные числа: ", eigenvalues )

Собственные векторы:  [[ 0.01825587+0.40302727j  0.01825587-0.40302727j  0.15505197+0.j
   0.03388407+0.j        ]
 [ 0.19846049+0.16674582j  0.19846049-0.16674582j  0.59845881+0.j
   0.07382927+0.j        ]
 [ 0.60643312-0.03586319j  0.60643312+0.03586319j  0.75083704+0.j
   0.99668772+0.j        ]
 [-0.6332438 +0.j         -0.6332438 -0.j         -0.23248585+0.j
   0.00383501+0.j        ]]
Собственные числа:  [-3.57411344+17.93783582j -3.57411344-17.93783582j
 11.85192949 +0.j          2.76629739 +0.j        ]
